In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer

from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


In [107]:
def evalua_metodo(modelo, X_train, y_train, X_test, y_test, mostrar=False):
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    modelo.fit(X_train, y_train)
    
    

    # Realizar predicciones en los datos de prueba
    y_pred = modelo.predict(X_test)
    y_proba = modelo.predict_proba(X_test)[:, 1]

    # Calcula métricas
    matriz_confusion = confusion_matrix(y_test, y_pred)
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    score_auc = auc(fpr, tpr)
    accuracy = accuracy_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    # Mostrar la Curva ROC
    if mostrar:
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f"AUC = {score_auc:.2f}")
        plt.plot([0, 1], [0, 1], 'r--')
        plt.xlabel('FPR')
        plt.ylabel('TPR')
        plt.title('ROC Curve')
        plt.legend()
        plt.show()

    # Devolver las métricas en un diccionario
    return { 'Matriz de Confusión': matriz_confusion, 'AUC': score_auc,
        'Accuracy': accuracy, 'MSE': mse
    }

def cross_validation(modelo, k, X, y):
    kfold = KFold(n_splits=k, shuffle=True, random_state=42)
    resultados = {
        'Accuracy': [],
        'AUC': [],
        'MSE': []
    }

    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        resultado = evalua_metodo(modelo, X_train, y_train, X_test, y_test)
        
        # Agregar resultados de cada fold a la lista correspondiente
        resultados['AUC'].append(resultado['AUC'])
        resultados['Accuracy'].append(resultado['Accuracy'])
        resultados['MSE'].append(resultado['MSE'])

    # Calcular el promedio de cada métrica
    promedios = {key: np.mean(val) for key, val in resultados.items()}
    
    return promedios



In [111]:
# Cargar el conjunto de datos de cáncer de mama
data = load_breast_cancer()
X = data.data
y = data.target

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Instanciar los modelos
modelo_lda = LDA()
modelo_knn = KNeighborsClassifier(n_neighbors=5)
modelo_logistic = LogisticRegression()

# Evaluar LDA
print("KNN:", evalua_metodo(modelo_lda, X_train, y_train, X_test, y_test))
print("KNN CV:", cross_validation(modelo_lda, 5, X, y))

# Evaluar KNN
print("KNN:", evalua_metodo(modelo_knn, X_train, y_train, X_test, y_test))
print("KNN CV:", cross_validation(modelo_knn, 5, X, y))

# Evaluar Logistic Regression
print("Logistic Regression:", evalua_metodo(modelo_logistic, X_train, y_train, X_test, y_test))
print("Logistic Regression CV:", cross_validation(modelo_logistic, 5, X, y))


KNN: {'Matriz de Confusión': array([[ 56,   7],
       [  2, 106]]), 'AUC': 0.9907407407407407, 'Accuracy': 0.9473684210526315, 'MSE': 0.05263157894736842}
KNN CV: {'Accuracy': 0.9472442167365317, 'AUC': 0.9877519236729324, 'MSE': 0.0527557832634684}
KNN: {'Matriz de Confusión': array([[ 59,   4],
       [  1, 107]]), 'AUC': 0.9805261610817166, 'Accuracy': 0.9707602339181286, 'MSE': 0.029239766081871343}
KNN CV: {'Accuracy': 0.9507529886663562, 'AUC': 0.9864858906508879, 'MSE': 0.04924701133364384}
Logistic Regression: {'Matriz de Confusión': array([[ 60,   3],
       [  1, 107]]), 'AUC': 0.9979423868312757, 'Accuracy': 0.9766081871345029, 'MSE': 0.023391812865497075}
Logistic Regression CV: {'Accuracy': 0.9753920198726906, 'AUC': 0.9954850276788912, 'MSE': 0.02460798012730942}


In [109]:
def evalua_config(modelo_base, configs, k, X, y):
    mejor_config = None
    menor_error = 100000000
    resultados = []

    # Iterar sobre cada configuración de hiperparámetros
    for config in configs:
        # Crear una instancia del modelo con la configuración actual
        modelo = clone(modelo_base)
        modelo.set_params(**config)
        
        # Realizar validación cruzada
        res_cv = cross_validation(modelo, k, X, y)
        
        # Calcular el error promedio (MSE) de la validación cruzada
        mse_promedio = np.mean([r['MSE'] for r in res_cv])
        resultados.append((config, mse_promedio))
        
        # Actualizar la mejor configuración si el MSE actual es el menor
        if mse_promedio < menor_error:
            menor_error = mse_promedio
            mejor_config = config

    print("Resultados por configuración:")
    for conf, error in resultados:
        print(f"Configuración: {conf}, MSE Promedio: {error:.4f}")
    
    return mejor_config, menor_error

# Ejemplo de uso con regresión logística para buscar el λ óptimo en regularización
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone

# Configuraciones de prueba para el hiperparámetro C (inverso de λ)
c = np.logspace(-5,5,11)
configs = [{'penalty': 'l2', 'C': i} for i in c]

# Suponiendo que X_scaled y y están ya definidos y listos para usar
modelo_base = LogisticRegression(solver='lbfgs', max_iter=1000)
mejor_config, menor_error = evalua_config(modelo_base, configs, 5, X_scaled, y)
print("Mejor configuración:", mejor_config)
print("Menor error MSE:", menor_error)


TypeError: string indices must be integers, not 'str'

In [ ]:
def evalua_multiples_metodos(X, y):
    # Dividir los datos en entrenamiento, validación y prueba
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

    # Configuraciones para la regresión logística
    configs = [
        {'penalty': 'l2', 'C': 0.01},
        {'penalty': 'l2', 'C': 0.1},
        {'penalty': 'l2', 'C': 1},
        {'penalty': 'l2', 'C': 10},
        {'penalty': 'l2', 'C': 100}
    ]
    
    # Optimizar hiperparámetros para la regresión logística
    modelo_logistic = LogisticRegression(solver='lbfgs', max_iter=1000)
    mejor_config, _ = evalua_config(modelo_logistic, configs, 5, X_train, y_train)
    modelo_logistic.set_params(**mejor_config)
    
    # Evaluar la regresión logística optimizada en el conjunto de prueba
    resultados_logistic = evalua_metodo(modelo_logistic, X_train_val, y_train_val, X_test, y_test)
    
    # Evaluar LDA
    modelo_lda = LDA()
    resultados_lda = evalua_metodo(modelo_lda, X_train_val, y_train_val, X_test, y_test)
    
    # Evaluar KNN con k=3
    modelo_knn = KNeighborsClassifier(n_neighbors=3)
    resultados_knn = evalua_metodo(modelo_knn, X_train_val, y_train_val, X_test, y_test)
    
    # Preparar la tabla de resultados
    resultados = pd.DataFrame({
        'Modelo': ['Regresión Logística', 'LDA', 'KNN k=3'],
        'Configuración': [str(mejor_config), 'N/A', 'k=3'],
        'Accuracy': [resultados_logistic['Accuracy'], resultados_lda['Accuracy'], resultados_knn['Accuracy']],
        'AUC': [resultados_logistic['AUC'], resultados_lda['AUC'], resultados_knn['AUC']],
        'MSE': [resultados_logistic['MSE'], resultados_lda['MSE'], resultados_knn['MSE']]
    })
    
    return resultados

# Ejemplo de uso
# Suponiendo que X_scaled y y están ya definidos y listos para usar
resultados_modelos = evalua_multiples_metodos(X_scaled, y)
print(resultados_modelos)
